In [1]:
# %load 2.矩阵求解.py
import cv2
import numpy as np
import glob

# 设置寻找亚像素角点的参数，采用的停止准则是最大循环次数30和最大误差容限0.001
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 20, 0.001)
# 获取标定板角点的位置
objp = np.zeros((5*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:5].T.reshape(-1,2)  # 将世界坐标系建在标定板上，所有点的Z坐标全部为0，所以只需要赋值x和y
obj_points = []  # 存储3D点
img_points = []  # 存储2D点
images = glob.glob(r"input_biaoding/*.jpg")#读取用于标定的图像
i=0;
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    size = (gray.shape[1], gray.shape[0])
    ret, corners = cv2.findChessboardCorners(gray, (9, 5), None)#获取棋盘格角点

    if ret:
        obj_points.append(objp)
        corners2 = cv2.cornerSubPix(gray, corners, (5, 5), (-1, -1), criteria)  # 在原角点的基础上寻找亚像素角点
        if [corners2]:
            img_points.append(corners2)
        else:
            img_points.append(corners)

        cv2.drawChessboardCorners(img, (9, 5), corners, ret)  # 记住，OpenCV的绘制函数一般无返回值
        i+=1;
        cv2.imwrite('output_biaoding/conimg'+str(i)+'.jpg', img)
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(obj_points, img_points, (640,480), None, None)#参数矩阵求解
print("ret:", ret)
print("mtx:\n", mtx) # 内参数矩阵
print("dist:\n", dist)  # 畸变系数   distortion cofficients = (k_1,k_2,p_1,p_2,k_3)
print("rvecs:\n", rvecs)  # 旋转向量  # 外参数
print("tvecs:\n", tvecs ) # 平移向量  # 外参数

print("-----------------------------------------------------")

img = cv2.imread(images[2])
h, w = img.shape[:2]
newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))#显示更大范围的图片（正常重映射之后会删掉一部分图像）
print (newcameramtx)
print("------------------使用undistort函数-------------------")
dst = cv2.undistort(img,mtx,dist,None,newcameramtx)#基于图像验证标定是否ok
x,y,w,h = roi
dst1 = dst[y:y+h,x:x+w]
cv2.imwrite('output_biaoding/calibresult.jpg', dst1)
print ("方法一:dst的大小为:", dst1.shape)


ret: 0.22368528354784453
mtx:
 [[4.98678469e+04 0.00000000e+00 3.18481015e+02]
 [0.00000000e+00 3.99205134e+04 2.39203873e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
dist:
 [[-5.25760326e+01 -5.88466988e-01  3.22153024e-01 -7.78611285e-03
   2.38687870e-01]]
rvecs:
 (array([[-0.07039531],
       [ 0.4252715 ],
       [-0.07954638]]), array([[0.62190971],
       [0.62206092],
       [1.47033602]]), array([[-0.64324709],
       [ 0.01023798],
       [-3.06495137]]), array([[-0.62051278],
       [ 0.62130609],
       [-1.47605509]]), array([[-0.01864357],
       [ 0.40952383],
       [-0.0179378 ]]), array([[-0.48964292],
       [ 0.58327382],
       [-0.91652473]]), array([[-0.45740704],
       [ 0.57005107],
       [-0.8144572 ]]), array([[0.47404955],
       [0.57433341],
       [0.86776567]]), array([[0.40691752],
       [0.54345612],
       [0.68149641]]), array([[-0.6200762 ],
       [ 0.62309338],
       [-1.46753854]]))
tvecs:
 (array([[  -1.69387288],
       [  -4.64345